# Imports

In [1]:
import os
import pickle
from torch import nn
import torch
from openai import OpenAI
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModel, set_seed

# File Path Declaration

In [2]:
project_base_path = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))))
project_base_path

'/home/ANONYMOUS/projects/FALCON'

In [3]:
saved_v1_generated_data_path = os.path.join(project_base_path, "data/generation/snort/snort3-community-rules_v1.pkl")
saved_v1_generated_data_path

'/home/ANONYMOUS/projects/FALCON/data/generation/snort/snort3-community-rules_v1.pkl'

In [4]:
saved_v2_generated_data_path = os.path.join(project_base_path, "data/generation/snort/snort3-community-rules_v2.pkl")
saved_v2_generated_data_path

'/home/ANONYMOUS/projects/FALCON/data/generation/snort/snort3-community-rules_v2.pkl'

In [5]:
generated_rule_dir_path = os.path.join(project_base_path, "results/e2e/quantitative/snort/ibm-granite_33_8b_instruct")
generated_rule_dir_path

'/home/ANONYMOUS/projects/FALCON/results/e2e/quantitative/snort/ibm-granite_33_8b_instruct'

# Environment

In [7]:
SEED = 42
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
MAX_LEN = 512

model_path="/data/common/models/ibm-granite/granite-3.3-8b-instruct"
device=DEVICE
model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map=device,
        torch_dtype=torch.bfloat16,
    )
tokenizer = AutoTokenizer.from_pretrained(
        model_path
)

# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model_path,
#     model_kwargs={"torch_dtype": torch.bfloat16},
#     device_map=DEVICE,
# )

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
def get_granite_output(prompt: str) -> str:
    conv = [{"role": "user", "content":prompt}]
    input_ids = tokenizer.apply_chat_template(conv, return_tensors="pt", thinking=False, return_dict=True, add_generation_prompt=True).to(device)
    set_seed(SEED)
    output = model.generate(
        **input_ids,
        max_new_tokens=4096,
        
    )
    prediction = tokenizer.decode(output[0, input_ids["input_ids"].shape[1]:], skip_special_tokens=True)
    torch.cuda.empty_cache()
    
    return prediction

In [17]:
get_granite_output("Hello, which model version are you?")

'I am the latest version of the Granite model developed by IBM. How can I assist you today?'

# Helper Functions

In [18]:
def load_from_pickle(file_path) -> dict:
    """
    Loads data from a pickle file.

    :param file_path: Path to the pickle file
    :return: Loaded data
    """
    try:
        with open(file_path, 'rb') as file:
            return pickle.load(file)
    except Exception as e:
        print(f"Error loading data from pickle: {e}")
        return None

In [19]:
def get_first_n_elements(dictionary: dict, n: int) -> dict:
    """
    Get the first n elements of a dictionary.

    :param dictionary: The input dictionary
    :param n: The number of elements to retrieve
    :return: A dictionary with the first n elements
    """
    return dict(list(dictionary.items())[:n])

In [20]:
def save_string_as_txt(directory_path, file_name, content):
    """
    Saves a given string as a .txt file in the specified directory.

    Args:
        directory_path (str): Path to the directory where the file should be saved.
        file_name (str): Desired name of the file (with or without .txt extension).
        content (str): The string content to be written to the file.

    Returns:
        str: Full path to the saved file if successful, otherwise an empty string.
    """
    if not file_name.lower().endswith('.txt'):
        file_name += '.txt'
    
    file_path = os.path.join(directory_path, file_name)

    try:
        os.makedirs(directory_path, exist_ok=True)  # Create directory if it doesn't exist
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(content)
        return file_path
    except PermissionError:
        print(f"Error: Permission denied to write to '{file_path}'.")
    except Exception as e:
        print(f"An error occurred while saving the file: {e}")

    return ""

# Data Generation

In [21]:
# Load the data back from the pickle file
loaded_v1_data = load_from_pickle(saved_v1_generated_data_path)
print(len(loaded_v1_data.keys()))

4017


In [22]:
snort_cti_sample_dict = get_first_n_elements(loaded_v1_data, 10)

In [23]:
# Load the data back from the pickle file
loaded_v2_data = load_from_pickle(saved_v2_generated_data_path)
print(len(loaded_v2_data.keys()))

4017


In [24]:
snort_cti_sample_dict

{'alert tcp $HOME_NET 2589 -> $EXTERNAL_NET any ( msg:"MALWARE-BACKDOOR - Dagger_1.4.0"; flow:to_client,established; content:"2|00 00 00 06 00 00 00|Drives|24 00|",depth 16; metadata:ruleset community; classtype:misc-activity; sid:105; rev:14; )': '    Title: Detection of Dagger 1.4.0 Backdoor Activity Over TCP\n\n    Threat Category: Malware – Backdoor\n\n    Threat Name: Dagger 1.4.0\n\n    Detection Summary:\n\n    This signature is designed to detect network traffic associated with the Dagger 1.4.0 backdoor. The traffic is characterized by a specific sequence of bytes ("2|00 00 00 06 00 00 00|Drives|24 00|") found within the first 16 bytes of the data payload. This communication occurs from an infected internal host to an external destination and typically indicates unauthorized remote access capabilities.\n\n    Rule Metadata\n    Classification: Misc Activity\n\n    Ruleset: Community\n\n    Rule Logic Breakdown\n    Alert Type: alert\n\n    Protocol: tcp\n\n    Source IP: $HOME_

In [25]:
snorts, ctis = zip(*snort_cti_sample_dict.items())
snorts = list(snorts)
ctis = list(ctis)

In [26]:
len(snorts), len(ctis)

(10, 10)

In [27]:
def format_cti_snort_data_to_training_data(data: list[dict]) -> list[tuple]:
    """
    Format the CTI Snort data into training data.

    :param data: The data to format
    :return: Formatted training data
    """
    training_data = []
    for dataset in data:
        for key, value in dataset.items():
            training_data.append((key, value))
    return training_data

In [28]:
# Sample Dataset Format (list of (anchor, positive) sentence pairs)
full_dataset = format_cti_snort_data_to_training_data([loaded_v1_data, loaded_v2_data])
print(len(full_dataset))

8034


In [29]:
def remove_10_test_samples(training_data: list[tuple], test_pairs: dict) -> list[tuple]:
    # Extract all test keys and values into sets for quick lookup
    test_keys = set(test_pairs.keys())
    test_values = set(test_pairs.values())
    
    # Filter training data
    filtered_data = [(key, value) for key, value in training_data if key not in test_keys and value not in test_values]
    
    return filtered_data

In [30]:
# Sample Dataset Format (list of (anchor, positive) sentence pairs)
full_dataset = remove_10_test_samples(full_dataset, snort_cti_sample_dict)
print(len(full_dataset))

8014


In [31]:
# Split into training and testing sets (80% train, 20% test)
train_pairs, test_pairs = train_test_split(full_dataset, test_size=0.1, random_state=SEED)

In [33]:
len(test_pairs)

802

# Dataset Class

In [34]:
# Custom Dataset
class ContrastiveDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        anchor, positive = self.data[idx]
        encoded = self.tokenizer([anchor, positive], padding="max_length", truncation=True,
                                 max_length=MAX_LEN, return_tensors="pt")
        return {
            "input_ids_a": encoded["input_ids"][0],
            "attention_mask_a": encoded["attention_mask"][0],
            "input_ids_b": encoded["input_ids"][1],
            "attention_mask_b": encoded["attention_mask"][1],
        }

In [35]:
# Bi-Encoder Model
class SentenceEncoder(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(model_name)

    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        embeddings = outputs.last_hidden_state[:, 0]  # CLS token
        return nn.functional.normalize(embeddings, p=2, dim=1)  # Normalize for cosine similarity


# Evaluation Functions

In [36]:
def generate_rule_from_cti_prompt(input_cti: str) -> str:

  rule_generation_prompt = f"""

    You are a cybersecurity expert tasked with performing Snort rule generation for a given Cyber Threat Intelligence (CTI).
    There is a sample task input and output provided below.
    
    Sample CTI Input and corresponding Snort Output:

    CTI Input:
        
      Title: Detection of QAZ Worm Client Login Activity over TCP Port 7597

      Threat Category: Malware – Backdoor

      Threat Name: QAZ Worm

      Detection Summary:

      This signature is designed to detect network traffic associated with the QAZ Worm, specifically its client login activity. The worm exhibits characteristic behavior by initiating a connection and transmitting a unique identifier string (qazwsx.hsq) to a remote server over TCP port 7597. This communication typically indicates the presence of a backdoor that allows unauthorized access to infected systems.

      Rule Metadata
      Classification: Misc Activity

      Ruleset: Community

      Rule Logic Breakdown
      Alert Type: alert

      Protocol: tcp

      Source IP: $EXTERNAL_NET (any IP address outside the local trusted network)

      Source Port: any

      Destination IP: $HOME_NET (any IP address inside the local trusted network)

      Destination Port: 7597 (known port used by the QAZ worm)

      Flow: to_server, established
      (Traffic must be flowing to a server and part of an established connection)

      Content Match: "qazwsx.hsq"
      (String in the payload that identifies the worm’s presence)

      Message: "MALWARE-BACKDOOR QAZ Worm Client Login access"

      Technical Details
      Port 7597 is not a standard well-known port and is leveraged by the QAZ Worm for backdoor communications.

      The content string "qazwsx.hsq" is a unique identifier used by the worm’s client when connecting to a command-and-control server or to another infected host.

      Detection relies on the presence of this string within an established TCP session directed to a host on the internal network.

      Indicators of Compromise (IOCs)
      String Pattern: qazwsx.hsq

      Destination Port: 7597/tcp

      Recommended Actions
      Block or restrict traffic on port 7597 at the perimeter firewall.

      Investigate any internal systems that initiate or receive such connections.

      Perform malware scanning and forensic analysis on potentially compromised hosts.

      Update endpoint and network defense signatures to ensure coverage against this and similar threats.
      
    Snort Output:
        
      alert tcp $EXTERNAL_NET any -> $HOME_NET 7597 ( msg:"MALWARE-BACKDOOR QAZ Worm Client Login access"; flow:to_server,established; content:"qazwsx.hsq"; metadata:ruleset community; classtype:misc-activity; sid:108; rev:12; )


    Generate Snort from the provided CTI. Do not include anything that is not provided.
    Do not print anything like sure here is the CTI or anything else. Only print the CTI. 

    CTI Input: 
    
      {input_cti}

    Snort Output:

  """

  return rule_generation_prompt

# Test Code

In [37]:
test_ctis = [i[1] for i in test_pairs]
test_ctis[0]

'Title: Detection of Microsoft Compound File Binary v3 File Magic\n\nThreat Category: File Identification\n\nThreat Name: Microsoft Compound File Binary v3\n\nDetection Summary:\n\nThis signature is designed to detect network traffic associated with Microsoft Compound File Binary v3 files, specifically the file magic that identifies these files. The detection exhibits characteristic behavior by identifying a unique binary pattern within the file data transmitted over the network. This identification typically indicates the presence of a Microsoft Compound File Binary v3 file being transferred.\n\nRule Metadata\nClassification: Misc Activity\n\nRuleset: Community\n\nRule Logic Breakdown\nAlert Type: alert\n\nProtocol: tcp\n\nSource IP: $EXTERNAL_NET (any IP address outside the local trusted network)\n\nSource Port: $FILE_DATA_PORTS (common ports used for file transfers)\n\nDestination IP: $HOME_NET (any IP address inside the local trusted network)\n\nDestination Port: any\n\nFlow: to_cl

In [38]:
prompt = generate_rule_from_cti_prompt(test_ctis[0])
print(prompt)



    You are a cybersecurity expert tasked with performing Snort rule generation for a given Cyber Threat Intelligence (CTI).
    There is a sample task input and output provided below.
    
    Sample CTI Input and corresponding Snort Output:

    CTI Input:
        
      Title: Detection of QAZ Worm Client Login Activity over TCP Port 7597

      Threat Category: Malware – Backdoor

      Threat Name: QAZ Worm

      Detection Summary:

      This signature is designed to detect network traffic associated with the QAZ Worm, specifically its client login activity. The worm exhibits characteristic behavior by initiating a connection and transmitting a unique identifier string (qazwsx.hsq) to a remote server over TCP port 7597. This communication typically indicates the presence of a backdoor that allows unauthorized access to infected systems.

      Rule Metadata
      Classification: Misc Activity

      Ruleset: Community

      Rule Logic Breakdown
      Alert Type: alert

      

In [39]:
test_rule = get_granite_output(prompt)
print(test_rule)

alert tcp $EXTERNAL_NET $FILE_DATA_PORTS -> $HOME_NET any ( msg:"FILE-IDENTIFY Microsoft Compound File Binary v3 file magic detected"; flow:to_client,established; content:"|D0 CF 11 E0 A1 B1 1A E1|"; content:">|00 03 00|", within 4, distance 16; metadata:ruleset community; classtype:misc-activity; sid:109; rev:12; )


# Generate Rule from CTI

In [40]:
generated_rules = []

inference_counter = 0
for cti in tqdm(test_ctis, "Generating Snort rules from CTIs..."):
  prompt = generate_rule_from_cti_prompt(cti)
  rule = get_granite_output(prompt)
  generated_rules.append(rule)
  file_name = f"quantitative_eval_snort_rule_{inference_counter}.txt"
  save_string_as_txt(generated_rule_dir_path, file_name, rule)
  inference_counter += 1

Generating Snort rules from CTIs...: 100%|██████████| 802/802 [1:21:09<00:00,  6.07s/it]


In [41]:
len(generated_rules)

802

# CTI-Rule Semantic Evaluation

In [42]:
def compute_dot_product_matrix_batched(model, tokenizer, test_snorts, test_ctis, batch_size=64):
    # Tokenize snorts once (since all CTIs will be compared to them)
    tokenized_snorts = tokenizer(test_snorts, return_tensors="pt", padding=True, max_length=MAX_LEN, truncation=True)
    input_ids_snorts = tokenized_snorts["input_ids"].to(DEVICE)
    attention_mask_snorts = tokenized_snorts["attention_mask"].to(DEVICE)

    with torch.no_grad():
        emb_snorts = model(input_ids_snorts, attention_mask_snorts)  # (802, dim)
        emb_snorts = emb_snorts.detach()

    # Prepare output tensor for all dot products
    num_ctis = len(test_ctis)
    dot_product_matrix = []

    for i in range(0, num_ctis, batch_size):
        batch_ctis = test_ctis[i:i + batch_size]
        tokenized_ctis = tokenizer(batch_ctis, return_tensors="pt", padding=True, max_length=MAX_LEN, truncation=True)
        input_ids_ctis = tokenized_ctis["input_ids"].to(DEVICE)
        attention_mask_ctis = tokenized_ctis["attention_mask"].to(DEVICE)

        with torch.no_grad():
            emb_ctis = model(input_ids_ctis, attention_mask_ctis)  # (B, dim)
            dot_product_batch = torch.matmul(emb_ctis, emb_snorts.T)  # (B, 802)
            dot_product_matrix.append(dot_product_batch.cpu())

        # Cleanup
        del input_ids_ctis, attention_mask_ctis, emb_ctis, dot_product_batch
        torch.cuda.empty_cache()

    # Concatenate batches into full matrix
    dot_product_matrix = torch.cat(dot_product_matrix, dim=0)  # (802, 802)
    return dot_product_matrix


In [43]:
RUN = 2
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
FINE_TUNED_MODEL_NAME = "all-mpnet-base-v2"
MODEL_NAME = f"/data/common/models/sentence-transformers/{FINE_TUNED_MODEL_NAME}"
FINE_TUNED_MODEL_STATE_NAME = f"contrastive_encoder_r{RUN}.pt"
MODEL_SAVE_PATH = os.path.join(project_base_path, f"script/fine_tuning/bi-encoder/snort/{FINE_TUNED_MODEL_NAME}/{FINE_TUNED_MODEL_STATE_NAME}")


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Load model
model = SentenceEncoder(MODEL_NAME).to(DEVICE)
model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))

<All keys matched successfully>

In [44]:
dot_product_matrix_test = compute_dot_product_matrix_batched(
    model=model,
    tokenizer=tokenizer,
    test_snorts=generated_rules,
    test_ctis=test_ctis,
    batch_size=256
)

In [45]:
import numpy as np

def extract_sigmoid_diagonal(dot_product_matrix: torch.Tensor):
    """
    Extracts principal diagonal from a dot-product matrix,
    applies sigmoid to each value, and returns the sigmoid list.
    Also prints the mean of sigmoid values.

    Args:
        dot_product_matrix (torch.Tensor): A square matrix of shape [N x N]

    Returns:
        List[float]: Sigmoid values of diagonal entries
    """
    assert dot_product_matrix.shape[0] == dot_product_matrix.shape[1], "Matrix must be square."

    # Step 1: Extract diagonal
    diag_values = dot_product_matrix.diag()  # shape: (N,)

    # Step 2: Apply sigmoid
    sigmoid_values = torch.sigmoid(diag_values)

    # Step 3: Convert to list and compute mean
    sigmoid_list = sigmoid_values.tolist()
    mean_value = torch.mean(sigmoid_values).item()

    # Output
    print(f"Mean of sigmoid(diagonal values): {mean_value:.4f}")
    return sigmoid_list

In [46]:
sigmoid_diagonal_scores = extract_sigmoid_diagonal(dot_product_matrix_test)

Mean of sigmoid(diagonal values): 0.7208


# Ragas 

In [47]:
open_ai_key = "OPENAI_KEY"
os.environ['OPENAI_API_KEY'] = open_ai_key

In [48]:
import numpy as np
from langchain.embeddings import OpenAIEmbeddings

def compute_dot_product_matrix_openai(test_snorts, test_ctis, batch_size=50):
    embedder = OpenAIEmbeddings()  # Uses text-embedding-ada-002 by default

    # Step 1: Get embeddings for all Snort rules
    snort_embeddings = []
    for i in range(0, len(test_snorts), batch_size):
        batch = test_snorts[i:i + batch_size]
        snort_embeddings.extend(embedder.embed_documents(batch))  # List of vectors

    snort_embeddings = np.array(snort_embeddings)  # Shape: (N, D)
    snort_embeddings_norm = np.linalg.norm(snort_embeddings, axis=1, keepdims=True)

    # Step 2: Compute batched dot products with CTIs
    dot_product_matrix = []

    for i in range(0, len(test_ctis), batch_size):
        batch = test_ctis[i:i + batch_size]
        cti_embeddings = embedder.embed_documents(batch)
        cti_embeddings = np.array(cti_embeddings)
        cti_embeddings_norm = np.linalg.norm(cti_embeddings, axis=1, keepdims=True)

        # Normalize and compute dot product
        sim_matrix = np.dot(cti_embeddings, snort_embeddings.T) / (
            cti_embeddings_norm @ snort_embeddings_norm.T
        )
        dot_product_matrix.append(sim_matrix)

    dot_product_matrix = np.vstack(dot_product_matrix)  # Final shape: (len(test_ctis), len(test_snorts))
    return dot_product_matrix

In [49]:
def extract_diagonal(dot_product_matrix: torch.Tensor):
    """
    Extracts principal diagonal from a dot-product matrix,
    applies sigmoid to each value, and returns the sigmoid list.
    Also prints the mean of sigmoid values.

    Args:
        dot_product_matrix (torch.Tensor): A square matrix of shape [N x N]

    Returns:
        List[float]: Sigmoid values of diagonal entries
    """
    assert dot_product_matrix.shape[0] == dot_product_matrix.shape[1], "Matrix must be square."

    # Step 1: Extract diagonal
    diag_values = dot_product_matrix.diag()  # shape: (N,)

    # Step 3: Convert to list and compute mean
    diag_list = diag_values.tolist()
    mean_value = torch.mean(diag_values).item()

    # Output
    print(f"Mean of sigmoid(diagonal values): {mean_value:.4f}")
    return diag_list

In [50]:
dot_product_matrix_test = compute_dot_product_matrix_openai(
    test_snorts=generated_rules,
    test_ctis=test_ctis,
    batch_size=50
)

/tmp/ipykernel_842383/1477236462.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedder = OpenAIEmbeddings()  # Uses text-embedding-ada-002 by default


In [51]:
diagonal_scores_openai = extract_diagonal(torch.tensor(dot_product_matrix_test))

Mean of sigmoid(diagonal values): 0.8797


# Bert

In [52]:
from bert_score import score

def compute_bert_scores(cti_list, generated_rules, lang="en", model_type="bert-base-uncased"):
    """
    Computes BERTScore between CTI descriptions and generated Snort rules.

    Args:
        cti_list (List[str]): List of CTI strings.
        generated_rules (List[str]): Corresponding generated Snort rule strings.
        lang (str): Language (default = "en").
        model_type (str): BERT model to use (default = DeBERTa-MNLI).

    Returns:
        dict: Precision, Recall, F1 scores (averaged) and all individual F1s.
    """
    assert len(cti_list) == len(generated_rules), "Mismatch in CTI and rule count."

    P, R, F1 = score(generated_rules, cti_list, lang=lang, model_type=model_type, verbose=True)

    mean_p = P.mean().item()
    mean_r = R.mean().item()
    mean_f1 = F1.mean().item()

    print(f"\nBERTScore Results:\nPrecision: {mean_p:.4f} | Recall: {mean_r:.4f} | F1: {mean_f1:.4f}")
    return {
        "precision": mean_p,
        "recall": mean_r,
        "f1": mean_f1,
        "f1_scores": F1.tolist()  # optional: return individual F1s
    }


In [53]:
bert_score_results = compute_bert_scores(test_ctis, generated_rules)

calculating scores...
computing bert embedding.


  0%|          | 0/25 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/13 [00:00<?, ?it/s]

done in 13.40 seconds, 59.85 sentences/sec

BERTScore Results:
Precision: 0.7038 | Recall: 0.5473 | F1: 0.6155
